<a href="https://colab.research.google.com/github/nstsj/Master_thesis_additional/blob/master/Appendix6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

текст-введение

In the first case the system may allow the semantically close types of entities or relations. 

For example, in the sentence “У меня уже полгода временами болит зуб, но сегодня заболело еще и горло”, the piece “болит зуб” can be denoted as an entity of class “symptom”, “complaint” or “anamnesis”. 

In this case the extracted class (c) should be counted as correct if it belongs to a set of possible classes C:

```Correct(X) = {1 if c(X) ∈ C, 0 otherwise``` <br>
```C ={c1,c2,...,cn} , C ∈ N```


If a more sophisticated solution is required, an additional step may be included to calculate label probability. In this case the sum of all class probabilities should result in 1.
 
$\sum_{i=1}^{n} x_i$
